# **Data Collection**

## Objectives

- Fetch data from Kaggle, save as raw data and prepare it for further processes.

## Inputs

- Kaggle JSON file - the token is required for kaggle authentication. 

## Outputs

- Generate the Dataset: inputs/cherry_leaves_dataset.

## Additional Comments

- No additional comments here.



---

# Import packages

In [1]:
! pip install -r /workspace/mildew-detector-pp5/requirements.txt

In [2]:
import numpy
import os
import random

# Change the working directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspace/mildew-detector-pp5/jupyter_notebooks'

In [4]:

os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


In [5]:
current_dir = os.getcwd()
current_dir

'/workspace/mildew-detector-pp5'

# Install Kaggle

In [6]:
! pip install kaggle

Run the cell below to change kaggle configuration directory to current working directory and permission of kaggle authentication json

In [7]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()

! chmod 600 kaggle.json


* Get the dataset path from the [Kaggle url](https://www.kaggle.com/datasets/codeinstitute/cherry-leaves).
* Set your destination folder.


Set the Kaggle Dataset and Download it.

In [8]:
KaggleDatasetPath = "codeinstitute/cherry-leaves/data"
DestinationFolder = "inputs/cherry_leaves_dataset"   
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

 98%|█████████████████████████████████████▎| 54.0M/55.0M [00:02<00:00, 26.3MB/s]
100%|██████████████████████████████████████| 55.0M/55.0M [00:02<00:00, 22.8MB/s]


Unzip the downloaded file, and delete the zip file.

In [9]:
import zipfile
with zipfile.ZipFile(DestinationFolder + '/cherry-leaves.zip', 'r') as zip_ref:
    zip_ref.extractall(DestinationFolder)

os.remove(DestinationFolder + '/cherry-leaves.zip')

---

# Data Preparation

---

## Data cleaning

### Check files, remove all non-image 

In [10]:
def remove_non_image_file(my_data_dir):
    """
    Removes non-image files from the dataset directories.
    """
    image_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.gif')  # Add other extensions if needed
    removed_files_count = 0
    
    for folder in os.listdir(my_data_dir):
        folder_path = os.path.join(my_data_dir, folder)
        if os.path.isdir(folder_path):  # Ensure it’s a directory
            for given_file in os.listdir(folder_path):
                file_location = os.path.join(folder_path, given_file)
                if os.path.isfile(file_location) and not given_file.lower().endswith(image_extensions):
                    os.remove(file_location)  # Remove non-image file
                    removed_files_count += 1

    print(f"Removed {removed_files_count} non-image files.")



In [11]:
remove_non_image_file(my_data_dir='inputs/cherry_leaves_dataset/cherry-leaves')

Removed 0 non-image files.


---

# Split Train Validation Test set


In [12]:
import os
import shutil
import random
import joblib

In [13]:
def split_train_validation_test_images(my_data_dir, train_set_ratio, validation_set_ratio, test_set_ratio):

    if train_set_ratio + validation_set_ratio + test_set_ratio != 1.0:
      print("train_set_ratio + validation_set_ratio + test_set_ratio should sum to 1.0")
      return

    labels = os.listdir(my_data_dir)
    if 'test' in labels:
      pass
    else:
        for folder in ['train', 'validation', 'test']:
            for label in labels:
                os.makedirs(name=my_data_dir + '/' + folder + '/' + label)

        for label in labels:

            files = os.listdir(my_data_dir + '/' + label)
            random.shuffle(files)

            train_set_files_qty = int(len(files) * train_set_ratio)
            validation_set_files_qty = int(len(files) * validation_set_ratio)

            count = 1
            for file_name in files:
                if count <= train_set_files_qty:
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/train/' + label + '/' + file_name)

                elif count <= (train_set_files_qty + validation_set_files_qty):
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/validation/' + label + '/' + file_name)

                else:
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/test/' + label + '/' + file_name)

                count += 1

            os.rmdir(my_data_dir + '/' + label)

The ratio that the data will be split into for each set:
* Train set is divided into a 0.70 ratio.
* Validation set is divided into a 0.10 ratio.
* Test set is divided into a 0.20 ratio.

In [14]:
split_train_validation_test_images(my_data_dir = f"inputs/cherry_leaves_dataset/cherry-leaves",
                                   train_set_ratio=0.7,
                                   validation_set_ratio=0.1,
                                   test_set_ratio=0.2
                                   )

---

# Conclusions and Next Steps 
* The image data has been collected, cleaned and split into train, validation and test sets.
* Next step: Data Visualization.